In this note book we will price a vanilla call option using QuantLib. The payoff of the option is plain vanilla (S_t - K). We price differnt option exercise types - European, American and Bermudan. We will price the option using different pricing methods - analytic, binomial tree, finite differential and monte carlo. We also vary the underlying process between bsm and heston.

Download QuantLib from Anaconda prompt using: pip install QuantLib-Python.
Next import Quantlib.

In [1]:
import QuantLib as ql
import matplotlib.pyplot as plt
%matplotlib inline

The option data is set here:

In [2]:
# option data
maturity_date = ql.Date(15, 1, 2020)
spot_price = 127.62
strike_price = 130
volatility = 0.20 # the historical vols for a year
dividend_rate =  0.0163
option_type = ql.Option.Call #CallOrPut

risk_free_rate = 0.001
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

calculation_date = ql.Date(1, 3, 2019) 
ql.Settings.instance().evaluationDate = calculation_date

Construct the option by setting payoff (S_t-K) and exercise type (european, american or bermudan).

In [4]:
# construct the European Option, American Option
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
settlement = calculation_date
bermuda_dates = ql.Schedule(ql.Date(1, 3, 2019), maturity_date, ql.Period(ql.Monthly), calendar, ql.Following,
                           ql.Following, ql.DateGeneration.Forward, False)

exercise_eu = ql.EuropeanExercise(maturity_date) 
exercise_am = ql.AmericanExercise(settlement, maturity_date)
exercise_bm = ql.BermudanExercise(list(bermuda_dates))
european_option = ql.VanillaOption(payoff, exercise_eu)
american_option = ql.VanillaOption(payoff, exercise_am)
bermudan_option = ql.VanillaOption(payoff, exercise_bm)

Setting up the process. Here we are using BSM. We can also use Heston process.

In [5]:
#BSM Process
spot_handle = ql.QuoteHandle(
    ql.SimpleQuote(spot_price)
)

flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count)
)

dividend_yield = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_rate, day_count)
)

flat_vol_ts = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
)

bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                           dividend_yield, 
                                           flat_ts, 
                                           flat_vol_ts)
#HestonProcess
"""
In order to create the Heston process, we use the parameter values: mean reversion strength kappa = 0.1, the
spot variance v0 = volatility*volatility = 0.04, the mean reversion variance theta=v0,
volatility of volatility sigma = 0.1 and the correlation between the asset price and its variance is
rho = -0.75
"""
v0=volatility*volatility #Spot Variance
kappa = 0.1
theta = v0
sigma = 0.1
rho = -0.75

heston_process = ql.HestonProcess(flat_ts, dividend_yield,spot_handle,
                                  v0, kappa, theta, sigma, rho)

The process can be priced using different pricing engines. Here we are performed using "AnalyticEuropeanEngine".

In [6]:
engine = ql.AnalyticEuropeanEngine(bsm_process)

european_option.setPricingEngine(engine)
bs_price = european_option.NPV()
print ("The theoretical price of european option is" , bs_price)

h_engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process), 0.01, 1000)
european_option.setPricingEngine(h_engine)
h_price = european_option.NPV()
print ("The heston model price for european option is" , h_price)

The theoretical price of european option is 7.636460507698542
The heston model price for european option is 7.352069734775412


Pricing using Binomial Tree

In [7]:
def binomial_price(option, bsm_process, steps):
    binomial_engine = ql.BinomialVanillaEngine(bsm_process, "crr", steps)
    option.setPricingEngine(binomial_engine)
    return option.NPV()

steps = 200 #range(2, 200, 1)

#BinomialTree-european option
bi_price = binomial_price(european_option, bsm_process, steps)
print ("The binomial tree price of european option is" , bi_price)

#BinomialTree-american option
bi_price = binomial_price(american_option, bsm_process, steps)
print ("The binomial tree price of american option is" , bi_price)

#BinomialTree-bermudan option
bi_price = binomial_price(bermudan_option, bsm_process, steps)
print ("The binomial tree price of bermudan option is" , bi_price)

The binomial tree price of european option is 7.644436140681128
The binomial tree price of american option is 7.76441381748751
The binomial tree price of bermudan option is 7.746982786230324


Pricing using Finite Differential method

In [8]:
#FDEngine-EuropeanOption
engine_FD = ql.FDEuropeanEngine(bsm_process,
                            timeSteps=100,
                            gridPoints=100,
                            timeDependent=False)
european_option.setPricingEngine(engine_FD)
FD_price = european_option.NPV()
print ("The FD price of European_Option_BSM is" , FD_price)

#FDEngine-AmericanOption
engine_FD = ql.FDAmericanEngine(bsm_process,
                            timeSteps=100,
                            gridPoints=100,
                            timeDependent=False)
american_option.setPricingEngine(engine_FD)
FD_price = american_option.NPV()
print ("The FD price of American_Option is" , FD_price)

#FDEngine-BermudanOption
engine_FD = ql.FDBermudanEngine(bsm_process,
                            timeSteps=100,
                            gridPoints=100,
                            timeDependent=False)
bermudan_option.setPricingEngine(engine_FD)
FD_price = bermudan_option.NPV()
print ("The FD price of Bermudan_Option is" , FD_price)


#FDHestonEngine-EuropeanOption
h_engine_FD = ql.FdHestonVanillaEngine(ql.HestonModel(heston_process))
european_option.setPricingEngine(h_engine_FD)
h_FD_price = european_option.NPV()
print ("The Heston FD price of European Option is" , h_FD_price)

#FDHestonEngine-AmericanOption
american_option.setPricingEngine(h_engine_FD)
h_FD_price = american_option.NPV()
print ("The Heston FD price of American Option is" , h_FD_price)

The FD price of European_Option_BSM is 7.647035245372445
The FD price of American_Option is 7.75144071693224
The FD price of Bermudan_Option is 7.750831840059611
The Heston FD price of European Option is 7.355566545831384
The Heston FD price of American Option is 7.485596639090531


Pricing using Monte Carlo method

In [9]:
#MCEngine-European Option
engine_MC = ql.MCEuropeanEngine(bsm_process, "PseudoRandom",
                            timeSteps=20,
                            requiredSamples=2500000)

european_option.setPricingEngine(engine_MC)
MC_price = european_option.NPV()
print ("The MC price of European Option is" , MC_price)

#MCEngine-American Option
engine_MC = ql.MCAmericanEngine(bsm_process, "PseudoRandom",
                            timeSteps=20,
                            requiredSamples=2500000)

american_option.setPricingEngine(engine_MC)
MC_price = american_option.NPV()
print ("The MC price of American Option is" , MC_price)

The MC price of European Option is 7.637921894675486
The MC price of American Option is 7.703999300955842
